In [ ]:
import pandas as pd

**Quoi investiguer**

Croissance par annee:
- Nombre de Events
- Athletes par quartier (seulement garder athlete <20 ans)
- Tranches d'age par annee
- Nombre d'athletes par genre
- Revenue
- Nombre de benevoles

Retention:
- 1 an, 2 ans, 3 ans
- Par age
- equipe/coach



### Reading the data

In [ ]:
df_2020 = pd.read_csv("/Users/emulie/Downloads/titane_registrations_2020.csv")
df_2021 = pd.read_csv("/Users/emulie/Downloads/titane_registrations_2021.csv")
df_2022 = pd.read_csv("/Users/emulie/Downloads/titane_registrations_2022.csv")
df_2023 = pd.read_csv("/Users/emulie/Downloads/titane_registrations_2023.csv")
df_2024 = pd.read_csv("/Users/emulie/Downloads/titane_registrations_2024.csv")

In [ ]:
# --- add year
df_2020['year'] = 2020
df_2021['year'] = 2021
df_2022['year'] = 2022
df_2023['year'] = 2023
df_2024['year'] = 2024

In [ ]:
df = pd.concat([df_2020, df_2021, df_2022, df_2023, df_2024])

In [ ]:
df['full_name'] = df['first_name'] + ' ' + df['last_name']

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
print(df.shape)

## EDA

### Number of Events by year

In [ ]:
df_events = df.groupby(['year'])['event_name'].nunique().reset_index().rename(columns={'event_name': 'num_events'})
df_events_name = df.groupby('year')['event_name'].agg(lambda x: list(set(x))).reset_index()
df_events = pd.merge(df_events, df_events_name, on='year')

In [ ]:
df_events

### Number of Player per year

In [ ]:
is_player_mask = df['age'] < 25
df_players = df[is_player_mask]
df_players['full_name'] = df_players['first_name'] + ' ' + df_players['last_name']
df_players_per_year = df_players.groupby(['year'])['full_name'].nunique().reset_index()

In [ ]:
df_players_per_year

### Number of player per gender by year

In [ ]:
has_valid_birthdate = df['birth_date'] != 'private'
is_valid_player = df['age'] < 25
df_ages = df[has_valid_birthdate & is_valid_player]
df_gender = df_ages.groupby(['year', 'gender']).count()['full_name'].reset_index(name='count')

In [ ]:
df_gender['total'] = df_gender.groupby('year')['count'].transform(sum)
df_gender['perc'] = df_gender['count'] / df_gender['total']

In [ ]:
df_gender

In [ ]:
# --- gender growth over year

### Revenue per Year

In [ ]:
df.groupby(['year'])['event_cost'].sum().reset_index()

### Age Distribution per Year

In [ ]:
has_valid_birthdate = df['birth_date'] != 'private'
is_valid_player = df['age'] < 25
df_ages = df[has_valid_birthdate & is_valid_player]
df_ages['full_name'] = df_ages['first_name'] + ' ' + df_ages['last_name']
df_ages['birth_date'] = pd.to_datetime(df_ages['birth_date'])
df_ages['age'] = df_ages['year'] - df_ages['birth_date'].dt.year

In [ ]:
df_age_distr_long = df_ages.groupby(['year', 'age'])['full_name'].nunique().reset_index()
df_age_distr_wide = pd.pivot_table(df_age_distr_long, index='year', values=['full_name'], columns=['age']).fillna(0).reset_index()

In [ ]:
df_age_distr_wide

In [ ]:
df_age_distr_long.head()

### Nombre de joueurs par arrondissement (TODO)

In [ ]:
df['locality'].unique()

In [ ]:
# --- TODO: map similar locality to the same

In [ ]:
df_locality_long = df.groupby(['year', 'locality'])['full_name'].nunique().reset_index(name='count')

In [ ]:
df_locality_long

In [ ]:
df_locality_wide = pd.pivot_table(df_locality_long, index='year', columns='locality', values='count').fillna(0)

In [ ]:
df_locality_wide

### Number of Benevoles (TODO)

## Retention Analysis

### Overall - How many year do player spend in Titane 

In [ ]:
df_active = df.groupby(['full_name'])['year'].nunique().reset_index(name='year_active')
df_active = df_active.groupby('year_active').count().reset_index().rename(columns={'full_name': 'count'})
df_active['total'] = df_active['count'].sum()
df_active['perc'] = df_active['count'] / df_active['total']

In [ ]:
df_active

### Retention per year (TODO)